#### Случайные числа

In [1]:
import os
os.urandom(16)

b'B \xa8\xec\xbe\x0e\xc8?Y\xa3\x90\x97:\xe9\x82\x81'

In [2]:
from secrets import token_bytes, token_hex, token_urlsafe
token_bytes(16)

b'K\xfb\xed\xd0\xf4\xf0\xfengm\x99z\xee\xe7PY'

In [3]:
token_hex(16)

'0c7230ec1ab00c6f7cb87eb610291c5c'

In [4]:
token_urlsafe(16)

'JlbkdUMCj-85xprAIphQYQ'

In [5]:
python -c 'import secrets; print(secrets.token_hex(16))'

SyntaxError: invalid syntax (1850908306.py, line 1)

#### Кодовые фразы

In [7]:
from pathlib import Path
import secrets
words = Path('/usr/share/dict/words').read_text().splitlines()
passphrase = ' '.join(secrets.choice(words) for i in range(4))
passphrase

FileNotFoundError: [Errno 2] No such file or directory: '/usr/share/dict/words'

#### Хеширование с ключом

In [8]:
from hashlib import blake2b
m = b'same message'
x = b'key x'
y = b'key y'
blake2b(m, key=x).digest() == blake2b(m, key=x).digest()

True

In [9]:
blake2b(m, key=x).digest() == blake2b(m, key=y).digest()

False

In [11]:
import hashlib
from pathlib import Path
def store(path, data, key):
    data_path = Path(path)
    hash_path = data_path.with_suffix('.hash')
    hash_value = hashlib.blake2b(data, key=key).hexdigest()
    with data_path.open(mode='x'), hash_path.open(mode='x'):
        data_path.write_bytes(data)
        hash_path.write_text(hash_value)
def is_modified(path, key):
    data_path = Path(path)
    hash_path = data_path.with_suffix('.hash')
    data = data_path.read_bytes()
    original_hash_value = hash_path.read_text()
    hash_value = hashlib.blake2b(data, key=key).hexdigest()
    return original_hash_value != hash_value

#### HMAC-функции

In [13]:
import hashlib
import hmac
hmac_sha256 = hmac.new(b'key', msg=b'message', digestmod=hashlib.sha256)

In [14]:
hmac_sha256.digest()

b"n\x9e\xf2\x9bu\xff\xfc[z\xba\xe5'\xd5\x8f\xda\xdb/\xe4.r\x19\x01\x19v\x91sC\x06_X\xedJ"

In [15]:
hmac_sha256.hexdigest()

'6e9ef29b75fffc5b7abae527d58fdadb2fe42e7219011976917343065f58ed4a'

In [16]:
hmac_sha256.digest_size

32

In [17]:
hmac_sha256.name

'hmac-sha256'

#### Проверка подлинности данных между системами

In [18]:
import hashlib
import hmac
import json
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
message = b'from Bob to Alice'
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
authenticated_msg = {
 'message': list(message),
 'hash_value': hash_value, }
outbound_msg_to_alice = json.dumps(authenticated_msg)

In [19]:
import hashlib
import hmac
import json
authenticated_msg = json.loads(inbound_msg_from_bob)
message = bytes(authenticated_msg['message'])
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
if hash_value == authenticated_msg['hash_value']:
    print('доверенное сообщение')

NameError: name 'inbound_msg_from_bob' is not defined

#### Атака по времени

In [20]:
from hmac import compare_digest
compare_digest('alice', 'mallory')

False

In [21]:
compare_digest('alice', 'alice')

True